<a href="https://colab.research.google.com/github/mohyunyang/My_Kaggle/blob/main/Aerial_Cactus_Identification_Step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 베이스라인모델

파이토치를 활용해 딥러닝 모델을 만들어보자

베이스라인은 CNN 활용

우선 시드값 고정 및 GPU 장비 설정을 해야한다

> 시드값 고정 

시드값고정 : 단순히 결과 재현을 위한 작업

GPU 장비 설정 : 훈련 속도를 높이기 위해 데이터를 GPU가 처리하도록 변경

> 데이터 준비 

훈련 / 검증 데이터 분리

데이터셋 클래스 정의 : 이미지데이터를 모델링에 적합한 형태로 불러오도록 해줌

데이터셋 생성 

데이터 로더 ( 데이터셋으로 부터 데이터를 배치 단위로 불러와주는 객체) 생성

> 모델 생성

신경망 모델 클래스를 직접 설계한 후 인스턴스 생성

> 모델 훈련

손실함수와 옵티마이저 설정 : 훈련에 앞서 손실 함수와 *옵티마이저 설정

모델훈련 : 신경망의 가중치(파라미터)를 갱신하며 모델 훈련

성능 검증 : 검증데이터로 모델 성능 검증

예측 및 제출 

`*최적의 가중치를 찾아주는 알고리즘`

## 1. 시드값 고정 및 GPU 장비 설정

딥러닝과 파이토치 특성 때문에 추가된 단계

> 시드값 고정 

In [ ]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                # 파이썬 난수 생성기 시드 고정
np.random.seed(seed)             # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)          # 파이토치 난수 생성기 시드 고정 (CPU 사용 시)
torch.cuda.manual_seed(seed)     # 파이토치 난수 생성기 시드 고정 (GPU 사용 시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정 (멀티GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

> GPU 장비 설정

이어서 장비를 설정해야한다.

정형데이터(csv파일 등)을 다루는 머신러닝과 달리 딥러닝은 주로 비정형데이터(이미지, 음성, 텍스트 등)을 다룬다.

비정형 데이터를 모델링하려면 연산량이 많아진다. 해서 CPU로는 감당하기 벅찰 정도라 훈련시간이 너무 길어진다.

그래서 GPU를 사용해야한다. GPU는 단순 연산 수백 수만개 이상을 병렬로 처리할 수 있어서 빠르게 훈련 가능하다.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 이렇게도 가능

In [ ]:
device

device(type='cpu')

In [ ]:
! pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yhyunmo","key":"8fdb7bbf0193b24d0178986819fcf478"}'}

# 2. 데이터준비

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle competitions download -c aerial-cactus-identification
! ls
! unzip aerial-cactus-identification.zip

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
 42% 5.00M/12.0M [00:00<00:00, 52.3MB/s]
100% 12.0M/12.0M [00:00<00:00, 102MB/s] 
aerial-cactus-identification.zip  kaggle.json  sample_data
Archive:  aerial-cactus-identification.zip
  inflating: sample_submission.csv   
  inflating: test.zip                
  inflating: train.csv               
  inflating: train.zip               


In [ ]:
import pandas as pd

labels = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')

In [ ]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile('train.zip') as zipper:
  zipper.extractall()

# 테스트 이미지 데이터 압축 풀기
with ZipFile('test.zip') as zipper:
  zipper.extractall()

> 훈련데이터 검증데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

In [ ]:
print('훈련 데이터 개수 : ', len(train))
print('검증 데이터 개수 : ', len(valid))

훈련 데이터 개수 :  15750
검증 데이터 개수 :  1750


> 데이터셋 클래스 정의

파이토치에서 제공하는 Dataset 클래스를 활용해 클래스 객체를 만들 수 있다

Dataset은 추상 클래스이며 우리는 Dataset을 상속받은 다음 특수 메서드인 "__len__"()과 __getitem__() 을 재정의(오버라이딩) 해야 한다.

In [ ]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__() # 상속받은 Dataset의 생성자 호출
        # 전달받은 인수들 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터셋 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]    # 이미지 ID
        img_path = self.img_dir + img_id # 이미지 파일 경로 
        image = cv2.imread(img_path)     # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1]     # 이미지 레이블(타깃값)

        if self.transform is not None:
            image = self.transform(image) # 변환기가 있다면 이미지 변환
        return image, label


> 데이터셋 생성

In [ ]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

torchvision은 파이토치용 컴퓨터 비전 라이브러리다

transforms 은 다양한 이미지 변환기를 제공하는 모듈이다

ToTensor() 메서드로 이미지를 텐서로 바꿨다. 이때 (가로픽셀 수, 세로픽셀 수, 채널 수) 형상이 (채널 수, 가로필셀 수, 세로픽셀 수) 형상으로 바뀜

파이토치르 이미지처리 할 때는 형상이 (채널 수, 가로필셀 수, 세로픽셀 수) 이여야 함

32 x 32 x 3 => 3 x 32 x 32 로 바뀐다는 뜻

여기에 배치가 추가되면 (배치크기, 채널 수, 가로필셀 수, 세로픽셀 수)

In [ ]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

> 데이터 로더 생성

데이터셋 다음으로는 데이터로더를 생성해야한다

데이터로더는 지정한 배치크기만큼 데이터를 불러오는 객체이다

딥러닝모델을 훈련할 때는 주로 단위로 데이터를 가져와 훈련한다

묶음단위로 훈련하는 게 훨씬 빠르게 때문이다

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

- dataset : 앞서 만든 이미지 데이터셋을 전달한다
- batch_size : 배치크기다. 훈련데이터와 검증 데이터 모두 한 번에 32개씩 불러오게 설정함 ( 4~ 256 까지 다양함, 그외 값도 가능, 배치크게에 정답은 없다)
- shuffle : 데이터를 섞을지 여부를 결정한다. 특정 데이터가 몰려 있을 경우를 대비, 훈련데이터에는 True 를 전달해 데이터를 섞었다

# 3. 모델생성

이번 절에는 기본적인 합성곱 신경망 CNN 모델을 만들어보겠다
(3, 32, 32) 형상의 이미지 데이터를 두 번의 합성곱과 풀링, 평탄화, 전결합 등을 거쳐 최종적으로 값이 0일 확률과 1일 확률을 구할 것이다

(3,32,32) ▶ 합성곱 ▶ 최대풀링 ▶ 합성곱 ▶ 최대풀링 ▶ 평균풀링 ▶ 평탄화 ▶ 전결합 ▶ 출력(0일확률/1일확률)

In [ ]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

class Model(nn.Module):
    # 신경망 계층 정의 
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        
        # 첫 번째 합성곱 계층 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=3, padding=2) 
        # 두 번째 합성곱 계층 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                               kernel_size=3, padding=2) 
        # 최대 풀링 계층 
        self.max_pool = nn.MaxPool2d(kernel_size=2) 
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층 
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의 
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

> Model 클래스를 정의하는 또 다른 방법

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__() 
        # 첫 번째 합성곱, 최대 풀링 계층
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, 
                                              out_channels=32, 
                                              kernel_size=3, 
                                              padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # 두 번째 합성곱, 최대 풀링 계층
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, 
                                              out_channels=64, 
                                              kernel_size=3, 
                                              padding=2),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # 평균 풀링 계층
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

In [ ]:
model = Model().to(device)

model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

# 모델훈련

> 손실함수 설정

신경망 모델훈련은 가중치를 갱신하는 작업이다 

예측값과 실제값의 손실이 작아지는 방향으로 이루어진다

손실 함수로 교차엔트로피를 사용하겠다

In [ ]:
# 손실함수
criterion = nn.CrossEntropyLoss()

> 옵티마이저 설정

첫 번째 파라미터로 모델이 가진 파라미터들을 전달한다. (model.parameters())


In [ ]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

모델 훈련

- 1. 데이터 로더에서 배치 크기만큼 데이터를 불러온다
- 2. 불러온 이미지 데이터와 레이블(타깃값) 데이터를 장비(GPU 혹은 CPU)에 할당한다
- 3. 옵티마이저 내 기울기를 초기화한다
- 4. 신경망 모델에 입력데이터(이미지)를 전달해 순전파하여 출력값(예측값)을 구한다
- 5. 예측값과 실제 레이블(타깃값)을 비교해 손실을 계산한다.
- 6. 손실을 기반으로 역전파를 수행한다
- 7. 역전파로 구한 기울기를 활용해 가중치를 갱신한다
- 8. 1~7 절차를 반복횟수만큼 되풀이한다
- 9. 1~8 절파를 에폭만큼 반복한다

훈련데이터는 15,570개, 배치크기는 32로 설정했으니 

15,750 / 32 를 올림한 493번이다

In [ ]:
import math

math.ceil(len(train) / 32)

493

In [ ]:
len(loader_train) # 데이터로더는 하나의 배치를 한 묶음으로 처리하기 때문

493

> 모델훈련

In [ ]:
epochs = 10 # 총 에폭
# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복 
    for images, labels in loader_train: # 데이터로더의 길이 493번 만큼 반복 /loader_train로 부터 배치크기 32만큼의 이미지와 레이블추출해 변수할당
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device) # 훈련시 GPU 또는 CPU 활용하도록 함
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()  # 가 없다면 기존 기울기가 계속 누적됨 해서 초기화하는 이처럼 작업필요
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item() 
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step() # 새로운 가중치 = 기존가중치 - (학습률 x 기울기)
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')

에폭 [1/10] - 손실값: 0.1190
에폭 [2/10] - 손실값: 0.1188
에폭 [3/10] - 손실값: 0.1171
에폭 [4/10] - 손실값: 0.1118
에폭 [5/10] - 손실값: 0.1097
에폭 [6/10] - 손실값: 0.1050
에폭 [7/10] - 손실값: 0.1065
에폭 [8/10] - 손실값: 0.1022
에폭 [9/10] - 손실값: 0.1005
에폭 [10/10] - 손실값: 0.0984


# 성능검증

훈련이 끝났으니 검증데이터를 이용해 평가지표인 ROC AUC 값을 구해보자

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포드
# 실제값과 예측확률값을 담을 리스트 초기화
true_list = [] # 실제값을 담을 리스트
preds_list = [] # 예측값을 담을 리스트

> 모델 성능검증

In [ ]:
model.eval() # 모델을 평가상태로 설정

with torch.no_grad(): # 기울기 계산 비활성화(왜?)
  for images, labels in loader_valid: 
    # 이미지, 레이블데이터 미니배치를 장비에 할당
    images = images.to(device)
    labels = labels.to(device)

    # 순전파 : 이미지데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    outputs = model(images)
    preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측확률
    true = labels.cpu() # 실제값
    # 예측확률과 실제값을 리스트에 추가
    preds_list.extend(preds)
    true_list.extend(true)

# 검증데이터 ROC AUC 점수 계산
print(f'검증데이터 ROC AUC:, {roc_auc_score(true_list, preds_list)}')

검증데이터 ROC AUC:, 0.9930564283021238


In [ ]:
len(preds_list)

5250

In [ ]:
len(true_list)

5250

`preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]`

ROC AUC 를 구하려면 각 타깃값의 확률을 먼저 구해야한다

그래서 순전파 출력값인 outputs를 torch.softmax()에 넘겨 0일 확률과 1일 확률을 얻었다 

`outputs.cpu()` , `labels.cpu()`

위 함수는 각각 이전에 GPU에 할당했던 outputs과 labels의 데이터를 다시 CPU에 할당한다

그래야 roc_auc_socre() 함수로 ROC AUC값을 계산 할 수 있다

roc_auc_socre() 는 파이토치가 아니라 사이킷런 함수이므로 GPU에 있는 데이터를 직접사용하는데서 이유가 있다

# 예측 및 결과 제출

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform= transform)
loader_test = DataLoader(dataset= dataset_test, batch_size= 32, shuffle=False)

데이터에서 타깃확률이 1일 확률을 예측해보겠다

for문이 끝나면 preds 변수에 최종예측확률이 모두 저장되어 있을 것

In [ ]:
model.eval() # 모델을 평가상태로 설정

preds = [] # 타깃예측값 저장용 리스트 초기화

with torch.no_grad(): # 기울기 계산 비활성화
  for images, _ in loader_test:
    # 이미지데이터 미니배치를 장비에 할당
    images = images.to(device)

    # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
    outputs = model(images)
    # 타깃값이 1일 확률(예측값)
    preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist() # tolist() 를 하지않으면 제출불가 / 이는 텐서타입을 리스트로 바꾼 것
    # preds에 preds_part 이어붙이기 
    preds.extend(preds_part)

# 결과 제출

In [ ]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)